In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import HDBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.stats import gaussian_kde
import os
import itertools
import warnings
import math
import re

In [ ]:
embeddings_filename = 'all_colors_data_optimal_embeddings.csv'
results_name = 'all_colors_data'
base_path = r'C:\Users\shash\OneDrive\Documents\Dessertation\Astro-DIM\data\processed'
embeddings_filepath = r'C:\Users\shash\OneDrive\Documents\Dessertation\Astro-DIM\results\all colors data'
results_path = r'C:\Users\shash\OneDrive\Documents\Dessertation\Astro-DIM\results\all colors data\clustering'

warnings.filterwarnings('ignore')

PLOTS_PER_PAGE = 16

In [ ]:
print(f"Loading combined embeddings data from: {embeddings_filename}")
df_embeddings = pd.read_csv(os.path.join(embeddings_filepath, embeddings_filename))

print(f"Data shape: {df_embeddings.shape}")
print(f"Columns: {list(df_embeddings.columns)}")
print("\nFirst few rows:")
print(df_embeddings.head())

In [ ]:
sample_ids = df_embeddings['sample_id'].values

feature_cols = [col for col in df_embeddings.columns if col.startswith('feature_')]
features = df_embeddings[feature_cols].values if feature_cols else None

pca_cols = [col for col in df_embeddings.columns if col.startswith('PCA_')]
pca_data = df_embeddings[pca_cols].values if pca_cols else None

ae_cols = [col for col in df_embeddings.columns if col.startswith('AE_')]
ae_data = df_embeddings[ae_cols].values if ae_cols else None

print(f"\nDetected optimal embedding spaces:")
if features is not None:
    print(f"Original features: {features.shape} - {len(feature_cols)} features")
if pca_data is not None:
    print(f"Optimal PCA embeddings: {pca_data.shape} - {len(pca_cols)} components")
if ae_data is not None:
    print(f"Optimal AE embeddings: {ae_data.shape} - {len(ae_cols)} components")

In [ ]:
def run_hdbscan_all_combinations(X_data, space_name="", verbose=True):
    
    min_cluster_sizes = [5]
    min_samples_list = [5]
    cluster_selection_epsilons = [2.0]
    
    results = []
    
    if verbose:
        print(f"\nRunning HDBSCAN clustering for {space_name}")
        total_combinations = len(min_cluster_sizes) * len(min_samples_list) * len(cluster_selection_epsilons)
        print(f"Parameter combinations: {len(min_cluster_sizes)} × {len(min_samples_list)} × {len(cluster_selection_epsilons)} = {total_combinations}")
        print(f"Data shape: {X_data.shape}")
        print("-" * 80)
    
    combination_count = 0
    
    for min_cluster_size in min_cluster_sizes:
        for min_samples in min_samples_list:
            for cluster_selection_epsilon in cluster_selection_epsilons:
                combination_count += 1
                
                try:
                    clusterer = HDBSCAN(
                        min_cluster_size=min_cluster_size,
                        min_samples=min_samples,
                        metric='euclidean',
                        cluster_selection_method='eom',
                        cluster_selection_epsilon=cluster_selection_epsilon
                    )
                    
                    labels = clusterer.fit_predict(X_data)
                    
                    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                    n_noise = np.sum(labels == -1)
                    noise_fraction = n_noise / len(labels)
                    
                    result = {
                        'combination_id': combination_count,
                        'min_cluster_size': min_cluster_size,
                        'min_samples': min_samples,
                        'cluster_selection_epsilon': cluster_selection_epsilon,
                        'n_clusters': n_clusters,
                        'n_noise': int(n_noise),
                        'noise_fraction': noise_fraction,
                        'labels': labels.copy()
                    }
                    
                    results.append(result)
                    
                    if verbose:
                        print(f"{combination_count:2d}. mcs={min_cluster_size:>2} ms={min_samples:>2} eps={cluster_selection_epsilon:>3.1f} | clusters={n_clusters:>2} noise={noise_fraction:>5.1%}")
                
                except Exception as e:
                    if verbose:
                        print(f"{combination_count:2d}. Error with mcs={min_cluster_size} ms={min_samples} eps={cluster_selection_epsilon}: {e}")
                    continue
    
    if verbose:
        print("-" * 80)
        print(f"Completed {len(results)} successful clustering runs")
    
    results_data = []
    for result in results:
        results_data.append({
            'combination_id': result['combination_id'],
            'min_cluster_size': result['min_cluster_size'],
            'min_samples': result['min_samples'],
            'cluster_selection_epsilon': result['cluster_selection_epsilon'],
            'n_clusters': result['n_clusters'],
            'n_noise': result['n_noise'],
            'noise_fraction': result['noise_fraction']
        })
    
    if results_data:
        results_df = pd.DataFrame(results_data)
        space_clean = space_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
        results_filename = f'{results_name}_{space_clean}_hdbscan_results.csv'
        results_df.to_csv(os.path.join(results_path, results_filename), index=False)
        
        if verbose:
            print(f"Results saved to: {results_filename}")
    
    return results

In [ ]:
def create_cluster_specific_plots(X_data, labels, space_name, results_path, results_name, 
                                 feature_names=None, combination_info=None, plots_per_page=16):
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = np.sum(labels == -1)
    noise_fraction = n_noise / len(labels)
    
    if combination_info:
        param_text = f"mcs={combination_info['min_cluster_size']} ms={combination_info['min_samples']} eps={combination_info['cluster_selection_epsilon']}"
        combination_id = combination_info['combination_id']
    else:
        param_text = ""
        combination_id = "unknown"
    
    n_dims = X_data.shape[1]
    
    dim_pairs = list(itertools.combinations(range(n_dims), 2))
    total_pairs = len(dim_pairs)
    
    print(f"Total dimension pairs: {total_pairs}")
    print(f"Plots per page: {plots_per_page}")
    
    grid_cols = int(math.sqrt(plots_per_page))
    grid_rows = math.ceil(plots_per_page / grid_cols)
    
    pages = [dim_pairs[i:i+plots_per_page] for i in range(0, len(dim_pairs), plots_per_page)]
    
    print(f"Will create {len(pages)} pages with {grid_rows}x{grid_cols} grids")
    
    unique_labels = np.unique(labels)
    
    for cluster_label in unique_labels:
        if cluster_label >= 0:
            print(f"\nCreating plots for Cluster {cluster_label}...")
            for page_num, page_pairs in enumerate(pages, 1):
                create_single_cluster_page(X_data, labels, page_pairs, space_name, results_path, 
                                         results_name, feature_names, combination_info, 
                                         cluster_label, page_num, len(pages), grid_rows, grid_cols)
    
    if -1 in unique_labels:
        print(f"\nCreating plots for Noise points...")
        for page_num, page_pairs in enumerate(pages, 1):
            create_noise_page(X_data, labels, page_pairs, space_name, results_path, 
                            results_name, feature_names, combination_info, 
                            page_num, len(pages), grid_rows, grid_cols)


In [ ]:
def create_single_cluster_page(X_data, labels, dim_pairs, space_name, results_path, results_name,
                              feature_names, combination_info, cluster_label, page_num, total_pages, 
                              grid_rows, grid_cols):
    
    combination_id = combination_info['combination_id'] if combination_info else "unknown"
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_fraction = np.sum(labels == -1) / len(labels)
    param_text = f"mcs={combination_info['min_cluster_size']} ms={combination_info['min_samples']} eps={combination_info['cluster_selection_epsilon']}" if combination_info else ""
    
    cluster_mask = labels == cluster_label
    cluster_size = np.sum(cluster_mask)
    
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(4*grid_cols, 3*grid_rows))
    if len(dim_pairs) == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if len(dim_pairs) > 1 else [axes]
    
    main_title = f'{space_name} - CLUSTER {cluster_label} - Page {page_num}/{total_pages}\nCombo {combination_id}: {cluster_size} points | {param_text}'
    
    for idx, (dim1, dim2) in enumerate(dim_pairs):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        x, y = X_data[:, dim1], X_data[:, dim2]
        
        ax.scatter(x, y, c='lightgray', alpha=0.2, s=0.5, label='Other data')
        
        cluster_x, cluster_y = x[cluster_mask], y[cluster_mask]
        ax.scatter(cluster_x, cluster_y, c='red', alpha=0.8, s=3, label=f'Cluster {cluster_label}')
        
        if feature_names and len(feature_names) > max(dim1, dim2):
            xlabel = feature_names[dim1].replace('feature_', '')
            ylabel = feature_names[dim2].replace('feature_', '')
        else:
            xlabel = f'Dim {dim1+1}'
            ylabel = f'Dim {dim2+1}'
            
        ax.set_xlabel(xlabel, fontsize=8)
        ax.set_ylabel(ylabel, fontsize=8)
        ax.set_title(f'Dims {dim1+1}-{dim2+1}', fontsize=9)
        ax.grid(True, alpha=0.3)
    
    for idx in range(len(dim_pairs), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.suptitle(main_title, fontsize=10, y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    space_clean = space_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
    filename = f'{results_name}_{space_clean}_combo_{combination_id}_cluster_{cluster_label}_page_{page_num:02d}.png'
    plt.savefig(os.path.join(results_path, filename), dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Saved: {filename}")

In [ ]:
def create_noise_page(X_data, labels, dim_pairs, space_name, results_path, results_name,
                     feature_names, combination_info, page_num, total_pages, grid_rows, grid_cols):
    
    combination_id = combination_info['combination_id'] if combination_info else "unknown"
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_fraction = np.sum(labels == -1) / len(labels)
    param_text = f"mcs={combination_info['min_cluster_size']} ms={combination_info['min_samples']} eps={combination_info['cluster_selection_epsilon']}" if combination_info else ""
    
    noise_mask = labels == -1
    noise_size = np.sum(noise_mask)
    
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(4*grid_cols, 3*grid_rows))
    if len(dim_pairs) == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if len(dim_pairs) > 1 else [axes]
    
    main_title = f'{space_name} - NOISE POINTS - Page {page_num}/{total_pages}\nCombo {combination_id}: {noise_size} points | {param_text}'
    
    for idx, (dim1, dim2) in enumerate(dim_pairs):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        x, y = X_data[:, dim1], X_data[:, dim2]
        
        ax.scatter(x, y, c='lightgray', alpha=0.2, s=0.5, label='Other data')
        
        noise_x, noise_y = x[noise_mask], y[noise_mask]
        ax.scatter(noise_x, noise_y, c='black', alpha=0.6, s=1, label='Noise')
        
        if feature_names and len(feature_names) > max(dim1, dim2):
            xlabel = feature_names[dim1].replace('feature_', '')
            ylabel = feature_names[dim2].replace('feature_', '')
        else:
            xlabel = f'Dim {dim1+1}'
            ylabel = f'Dim {dim2+1}'
            
        ax.set_xlabel(xlabel, fontsize=8)
        ax.set_ylabel(ylabel, fontsize=8)
        ax.set_title(f'Dims {dim1+1}-{dim2+1}', fontsize=9)
        ax.grid(True, alpha=0.3)
    
    for idx in range(len(dim_pairs), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.suptitle(main_title, fontsize=10, y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    space_clean = space_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
    filename = f'{results_name}_{space_clean}_combo_{combination_id}_noise_page_{page_num:02d}.png'
    plt.savefig(os.path.join(results_path, filename), dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Saved: {filename}")

In [ ]:
def create_clustering_plots_all_pairs(X_data, labels, space_name, results_path, results_name, 
                                     feature_names=None, combination_info=None, plots_per_page=16):
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = np.sum(labels == -1)
    noise_fraction = n_noise / len(labels)
    
    if combination_info:
        param_text = f"mcs={combination_info['min_cluster_size']} ms={combination_info['min_samples']} eps={combination_info['cluster_selection_epsilon']}"
        combination_id = combination_info['combination_id']
    else:
        param_text = ""
        combination_id = "unknown"
    
    n_dims = X_data.shape[1]
    
    dim_pairs = list(itertools.combinations(range(n_dims), 2))
    total_pairs = len(dim_pairs)
    
    print(f"Total dimension pairs: {total_pairs}")
    print(f"Plots per page: {plots_per_page}")
    
    grid_cols = int(math.sqrt(plots_per_page))
    grid_rows = math.ceil(plots_per_page / grid_cols)
    
    pages = [dim_pairs[i:i+plots_per_page] for i in range(0, len(dim_pairs), plots_per_page)]
    
    print(f"Will create {len(pages)} pages with {grid_rows}x{grid_cols} grids")
    
    for page_num, page_pairs in enumerate(pages, 1):
        create_single_page_scatter(X_data, labels, page_pairs, space_name, results_path, 
                                 results_name, feature_names, combination_info, 
                                 page_num, len(pages), grid_rows, grid_cols)
    
    for page_num, page_pairs in enumerate(pages, 1):
        create_single_page_contour(X_data, labels, page_pairs, space_name, results_path, 
                                 results_name, feature_names, combination_info, 
                                 page_num, len(pages), grid_rows, grid_cols)


In [ ]:
def create_single_page_scatter(X_data, labels, dim_pairs, space_name, results_path, results_name,
                              feature_names, combination_info, page_num, total_pages, grid_rows, grid_cols):
    
    combination_id = combination_info['combination_id'] if combination_info else "unknown"
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_fraction = np.sum(labels == -1) / len(labels)
    param_text = f"mcs={combination_info['min_cluster_size']} ms={combination_info['min_samples']} eps={combination_info['cluster_selection_epsilon']}" if combination_info else ""
    
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(4*grid_cols, 3*grid_rows))
    if len(dim_pairs) == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if len(dim_pairs) > 1 else [axes]
    
    main_title = f'{space_name} - SCATTER - Page {page_num}/{total_pages}\nCombo {combination_id}: {n_clusters} Clusters, {noise_fraction:.1%} Noise | {param_text}'
    
    for idx, (dim1, dim2) in enumerate(dim_pairs):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        x, y = X_data[:, dim1], X_data[:, dim2]
        
        colors = labels.copy().astype(float)
        colors[labels == -1] = -0.5
        
        scatter = ax.scatter(x, y, c=colors, cmap='viridis', alpha=0.6, s=1)
        
        if feature_names and len(feature_names) > max(dim1, dim2):
            xlabel = feature_names[dim1].replace('feature_', '')
            ylabel = feature_names[dim2].replace('feature_', '')
        else:
            xlabel = f'Dim {dim1+1}'
            ylabel = f'Dim {dim2+1}'
            
        ax.set_xlabel(xlabel, fontsize=8)
        ax.set_ylabel(ylabel, fontsize=8)
        ax.set_title(f'Dims {dim1+1}-{dim2+1}', fontsize=9)
        ax.grid(True, alpha=0.3)
    
    for idx in range(len(dim_pairs), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.suptitle(main_title, fontsize=10, y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    space_clean = space_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
    filename = f'{results_name}_{space_clean}_combo_{combination_id}_scatter_page_{page_num:02d}.png'
    plt.savefig(os.path.join(results_path, filename), dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Saved: {filename}")

In [ ]:
def create_single_page_contour(X_data, labels, dim_pairs, space_name, results_path, results_name,
                              feature_names, combination_info, page_num, total_pages, grid_rows, grid_cols):
    
    combination_id = combination_info['combination_id'] if combination_info else "unknown"
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_fraction = np.sum(labels == -1) / len(labels)
    param_text = f"mcs={combination_info['min_cluster_size']} ms={combination_info['min_samples']} eps={combination_info['cluster_selection_epsilon']}" if combination_info else ""
    
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(4*grid_cols, 3*grid_rows))
    if len(dim_pairs) == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if len(dim_pairs) > 1 else [axes]
    
    main_title = f'{space_name} - CONTOUR - Page {page_num}/{total_pages}\nCombo {combination_id}: {n_clusters} Clusters, {noise_fraction:.1%} Noise | {param_text}'
    
    for idx, (dim1, dim2) in enumerate(dim_pairs):
        if idx >= len(axes):
            break
            
        ax = axes[idx]
        x, y = X_data[:, dim1], X_data[:, dim2]
        
        try:
            unique_labels = np.unique(labels[labels >= 0])
            colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))
            
            for i, label in enumerate(unique_labels):
                cluster_mask = labels == label
                cluster_points = np.sum(cluster_mask)
                
                cluster_x, cluster_y = x[cluster_mask], y[cluster_mask]
                
                if cluster_points < 3:
                    ax.scatter(cluster_x, cluster_y, c=[colors[i]], s=50, 
                                    alpha=0.8, marker='o', edgecolors='black', linewidth=0.5)
                    continue
                
                try:
                    if cluster_points <= 5:
                        bw_method = 1.0
                    elif cluster_points <= 10:
                        bw_method = 0.8
                    elif cluster_points <= 20:
                        bw_method = 0.6
                    else:
                        bw_method = 'scott'
                    
                    kde = gaussian_kde(np.vstack([cluster_x, cluster_y]), bw_method=bw_method)
                    
                    x_min, x_max = x.min() - 0.1*(x.max()-x.min()), x.max() + 0.1*(x.max()-x.min())
                    y_min, y_max = y.min() - 0.1*(y.max()-y.min()), y.max() + 0.1*(y.max()-y.min())
                    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 50), np.linspace(y_min, y_max, 50))
                    
                    density = kde(np.vstack([xx.ravel(), yy.ravel()])).reshape(xx.shape)
                    
                    if np.isfinite(density).all() and np.max(density) > np.min(density):
                        levels = 3 if cluster_points <= 10 else 5
                        ax.contour(xx, yy, density, levels=levels, 
                                        colors=[colors[i]], linewidths=1.0, alpha=0.7)
                    else:
                        ax.scatter(cluster_x, cluster_y, c=[colors[i]], s=20, 
                                        alpha=0.8, marker='^', edgecolors='black', linewidth=0.5)
                    
                except:
                    ax.scatter(cluster_x, cluster_y, c=[colors[i]], s=15, 
                                    alpha=0.8, marker='d', edgecolors='black', linewidth=0.5)
            
            if feature_names and len(feature_names) > max(dim1, dim2):
                xlabel = feature_names[dim1].replace('feature_', '')
                ylabel = feature_names[dim2].replace('feature_', '')
            else:
                xlabel = f'Dim {dim1+1}'
                ylabel = f'Dim {dim2+1}'
                
            ax.set_xlabel(xlabel, fontsize=8)
            ax.set_ylabel(ylabel, fontsize=8)
            ax.set_title(f'Dims {dim1+1}-{dim2+1}', fontsize=9)
            ax.grid(True, alpha=0.3)
            
        except Exception as e:
            ax.text(0.5, 0.5, 'Visualization\nnot available', ha='center', va='center', 
                          transform=ax.transAxes, fontsize=10, alpha=0.7)
            ax.set_title(f'Dims {dim1+1}-{dim2+1}', fontsize=9)
    
    for idx in range(len(dim_pairs), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.suptitle(main_title, fontsize=10, y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    space_clean = space_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
    filename = f'{results_name}_{space_clean}_combo_{combination_id}_contour_page_{page_num:02d}.png'
    plt.savefig(os.path.join(results_path, filename), dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Saved: {filename}")

In [ ]:
spaces = {}

if features is not None:
    spaces['Original Features'] = {
        'data': features,
        'feature_names': [col.replace('feature_', '') for col in feature_cols]
    }

if pca_data is not None:
    space_name = f'Optimal PCA Space ({pca_data.shape[1]}D)'
    spaces[space_name] = {
        'data': pca_data,
        'feature_names': pca_cols
    }

if ae_data is not None:
    space_name = f'Optimal AE Space ({ae_data.shape[1]}D)'
    spaces[space_name] = {
        'data': ae_data,
        'feature_names': ae_cols
    }

print(f"\nOptimal spaces to analyze: {list(spaces.keys())}")

In [ ]:
all_results = {}

for space_name, space_info in spaces.items():
    print(f"\n{'='*60}")
    print(f"ANALYZING: {space_name}")
    print(f"{'='*60}")
    
    X_data = space_info['data']
    feature_names = space_info['feature_names']
    
    if np.isnan(X_data).any():
        print(f"Warning: NaN values found in {space_name}, skipping...")
        continue
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_data)
    
    clustering_results = run_hdbscan_all_combinations(X_scaled, space_name, verbose=True)
    
    all_results[space_name] = clustering_results
    
    print(f"\nCreating plots for {len(clustering_results)} combinations...")
    
    for result in clustering_results:
        labels = result['labels']
        combination_info = {
            'combination_id': result['combination_id'],
            'min_cluster_size': result['min_cluster_size'],
            'min_samples': result['min_samples'],
            'cluster_selection_epsilon': result['cluster_selection_epsilon']
        }
        
        # Create cluster-specific plots 
        create_cluster_specific_plots(X_scaled, labels, space_name, results_path, 
                                    results_name, feature_names, combination_info, PLOTS_PER_PAGE)
        
        # Save cluster labels
        labels_df = pd.DataFrame({
            'sample_id': sample_ids,
            'cluster_labels': labels
        })
        
        space_clean = space_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
        labels_filename = f'{results_name}_{space_clean}_combo_{result["combination_id"]}_labels.csv'
        labels_df.to_csv(os.path.join(results_path, labels_filename), index=False)

print(f"\n{'='*60}")
print("CLUSTERING ANALYSIS COMPLETE")
print(f"{'='*60}")